# HuBMAP ASCT+B Augmentation with ARCHS4 Coexpression
This notebook contains the scripts used to create the HuBMAP ASCT+B Augmented with RNA-seq Coexpression dataset for Harmonizome. This uses the previously processed ASCT+B dataset, and submits each gene set from that dataset for augmentation using the [Geneshot](https://maayanlab.cloud/geneshot) API. This returns a list of genes that are correlated with the input genes based on a coexpression matrix calculated from [ARCHS4](https://maayanlab.cloud/archs4/).

In [ ]:
import pandas as pd
import datetime
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import requests
import time
import scipy.spatial.distance as dist
import seaborn as sns
import sys
import json
import scanpy as sc
from tqdm import tqdm

# UMAP
from sklearn.feature_extraction.text import TfidfVectorizer
import anndata
from collections import OrderedDict

# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, save, output_file
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

from IPython.display import display, HTML, Markdown
sys.setrecursionlimit(100000)

## Augment ASCT+B Gene Sets

In [ ]:
matrix = pd.read_csv('downloads/gene_attribute_matrix.txt.gz', sep='\t', compression='gzip', index_col=0)
matrix

In [ ]:
asctb_full_gmt = matrix.stack().reset_index().replace(0, pd.NA).dropna().groupby('level_1')['Gene'].agg(list).to_dict()
len(asctb_full_gmt)

### Query Geneshot API
For each cell type from the ASCT+B dataset, the gene set is sent to Geneshot's coexpression augmentation API endpoint, which returns a list of similar genes based on coexpression. Up to 99 genes are kept for each input gene set, such that ```len(input_gene_set) + len(augmented_genes) = 100```.

In [ ]:
GENESHOT_URL = 'https://maayanlab.cloud/geneshot/api/associate'

augment_gmt = {}
for term in tqdm(asctb_full_gmt):
    gene_list = list(asctb_full_gmt[term])
    payload = {
        "gene_list": gene_list,
        "similarity": "coexpression"
    }
    response = requests.post(GENESHOT_URL, json=payload)
    data = json.loads(response.text)

    sim_genes = pd.DataFrame(data['association']).T.sort_values('simScore', ascending=False)
    sim_genes# = sim_genes.head(100-len(gene_list))

    augment_gmt[term] = sim_genes['simScore'].to_dict()
    sleep(0.1)

len(augment_gmt)

In [ ]:
asctb_augmented = pd.DataFrame(index=augment_gmt.keys(), data=augment_gmt.values()).T.sort_index().rename_axis('Gene')
asctb_augmented.to_csv('asctb_geneshot_augmented_genes.tsv', sep='\t')
asctb_augmented

In [ ]:
pd.read_csv('asctb_geneshot_augmented_genes.tsv', sep='\t', index_col='Gene')

In [ ]:
asctb_augmented = asctb_augmented[asctb_augmented.index.isin(human_gene_info.index)]
asctb_augmented

In [ ]:
ax = sns.histplot(asctb_augmented.stack(), bins=100, binwidth=0.01)
plt.axvline(0.67, color='black')
ax.set_title('ASCT+B Augmentation Coexpression Scores')

In [ ]:
top = asctb_augmented[asctb_augmented>=0.68].count().sort_values(ascending=False)
top.describe()

In [ ]:
aug = {}
for cell in tqdm(asctb_full_gmt):
    aug[cell] = asctb_augmented[asctb_augmented>=0.68][cell].dropna().sort_values(ascending=False).index[:4*len(asctb_full_gmt[cell])].to_list()

aug

In [ ]:
sns.histplot(pd.Series(map(len,aug.values())), bins=100)

In [ ]:
asctb_augmented[cell][asctb_augmented[cell]>=0.67]

In [ ]:
fig, (ax0, ax1, ax2, ax3, ax4) = plt.subplots(1, 5, figsize=(24,4))
axs = [ax0, ax1, ax2, ax3, ax4]
#sns.histplot(pd.Series(map(len,asctb_full_gmt.values())), kde=True, ax=ax0)

for i in range(5):
    aug = {}
    for cell in tqdm(asctb_full_gmt):
        aug[cell] = asctb_augmented[cell][asctb_augmented[cell]>=0.67].dropna().sort_values(ascending=False).index[:i*len(asctb_full_gmt[cell])].to_list()

    aug

    augmented_gmt = {}
    for term in asctb_full_gmt:
        base = set(asctb_full_gmt[term])
        base.update(set(aug[term]))
        augmented_gmt[term] = base

    num_gmt_sets = sum(map(lambda x: len(x) >= 5, augmented_gmt.values()))
    sns.histplot(pd.Series(map(len,augmented_gmt.values())), kde=True, ax=axs[i])
    axs[i].axvline(4.5, 0, 360, color='black')
    axs[i].set_xlim(0, 50)
    axs[i].set_ylim(0, 360)
    axs[i].set_title(f'Cap = {i} * Set Length, n={num_gmt_sets}')
    axs[i].set_xlabel('Gene Set Length')

ax0.set_title('Unaugmented, n=350')

In [ ]:
ax = sns.scatterplot(x=pd.Series(map(len,asctb_full_gmt.values()), index=asctb_full_gmt.keys()), y=asctb_augmented[asctb_augmented>=0.68].stack().groupby(axis=0, level=1).count())
sns.lineplot(x=(0,16),y=(0,64), color='black', ax=ax)
ax.set_xlim(0,16)

In [ ]:
asctbaug = pd.Series(index=augmented_gmt.keys(), data=augmented_gmt.values()).explode().reset_index()
asctbaug.columns = ['Cell Type', 'Gene']
asctbaug

## Process Data for SQL Ingestion

### Dataset

In [ ]:
#(id, name, name_without_resource, description, association, gene_set_description, gene_sets_description, attribute_set_description, positive_association, negative_association, is_signed, is_continuous_valued, last_updated, directory, num_page_views, resource_fk, measurement_fk, dataset_group_fk, attribute_type_fk, attribute_group_fk, evidence_type, evidence_group, measurement_bias, attribute_type_plural)
(167, 'HuBMAP ASCT+B Augmented with RNA-seq Coexpression', 'ASCT+B Augmented with RNA-seq Coexpression', 'Anatomical structure and cell type biomarker annotations from the HuBMAP ASCT+B tables, augmented with RNA-seq coexpression data from ARCHS4', 'gene-cell type associations from curated genetic association studies', 'biomarker genes for the {0} cell type from the HuBMAP ASCT+B Augmented with RNA-seq Coexpression dataset.', 'sets of biomarker genes for cell types from the HuBMAP ASCT+B Augmented with RNA-seq Coexpression dataset.', 'cell types associated with {0} gene from the HuBMAP ASCT+B Augmented with RNA-seq Coexpression dataset.', 0, 0, '2025-01-29', 'asctbaugmented', 28, 111, 31, 6, 2, 1, 'association by genetic study curation', 'curated experimental data', 'mixed', 'cell types', '0'
)

### Genes

In [ ]:
human_gene_info = pd.read_csv('../../../mapping/source_files/human_gene_info', sep='\t')
human_gene_info = human_gene_info[human_gene_info['#tax_id']==9606]
human_gene_info['Symbol'] = human_gene_info['Symbol'].map(str.upper)
human_gene_info = human_gene_info[['Symbol', 'GeneID', 'description', 'type_of_gene']].set_index('Symbol')
human_gene_info

In [ ]:
gene_synonyms = human_gene_info.set_index('Symbol')['Synonyms'].apply(str.split, sep='|').explode().reset_index().set_index('Synonyms').to_dict()

In [ ]:
genes = pd.read_csv('../../../tables/gene.csv')
genes['symbol'] = genes['symbol'].apply(str.upper)
geneids = genes.set_index('symbol')['ncbi_entrez_gene_id'].to_dict()
genefks = genes.set_index('symbol')['id'].to_dict()

In [ ]:
gmtgenes = set()
for geneset in augmented_gmt.values():
    gmtgenes.update(geneset)
len(gmtgenes)

In [ ]:
for gene in gmtgenes:
    if gene.upper() not in geneids and '.' not in gene:
        if gene in human_gene_info.index:
            print(human_gene_info.loc[gene])
        else:
            print(human_gene_info.loc[gene_synonyms[gene]])

### Gene Set

In [ ]:
attributes = pd.read_csv('../../../tables/attribute.tsv', sep='\t')

attributes = attributes[attributes['naming_authority_fk']==105]
attributes['name_from_naming_authority'] = attributes['name_from_naming_authority'].map(str.upper)
attributes = attributes.set_index('name_from_naming_authority')

In [ ]:
genesetfks = {}
ctids = {}
index = 136800000
for celltype in asctbaug['Cell Type'].unique():
    print((index, celltype, attributes.loc[celltype.upper(), 'id_from_naming_authority'], 167, 2, attributes.loc[celltype.upper(), 'id']), end=',\n')
    genesetfks[celltype] = index
    ctids[celltype] = attributes.loc[celltype.upper(), 'id_from_naming_authority']
    index += 1

### Associations

In [ ]:
associations = asctbaug.copy()[['Gene', 'Cell Type']].rename_axis('id')
associations['Gene'] = associations['Gene'].map(str.upper).map(genefks)
associations['Cell Type'] = associations['Cell Type'].map(genesetfks)
associations.columns = ['gene_fk', 'gene_set_fk']
associations['threshold_value'] = 1
associations.index += 173000000
associations.to_csv('../../../harmonizome-update/asctbaugmented.csv')
associations

## Create Downloads

In [ ]:
output_path = 'augmenteddownloads/'

In [ ]:
asctbaug['Gene ID'] = asctbaug['Gene'].map(str.upper).map(geneids)
asctbaug['Cell Type ID'] = asctbaug['Cell Type'].map(ctids)
asctbaug = asctbaug[['Gene', 'Gene ID', 'Cell Type', 'Cell Type ID']]
asctbaug.columns = ['Gene', 'Gene ID', 'Cell Type', 'Cell Type ID']
asctbaug['Threshold'] = 1
asctbaug

### Gene Attribute Binary Matrix

In [ ]:
binarymatrix = pd.crosstab(asctbaug['Gene'], asctbaug['Cell Type'], asctbaug['Threshold'], aggfunc=max).replace(np.nan, 0).astype(int)
binarymatrixT = binarymatrix.T
binarymatrix.to_csv(output_path+'gene_attribute_matrix.txt.gz', sep='\t', compression='gzip')
binarymatrix

### Gene Attribute Edge List

In [ ]:
edgelist = asctbaug.copy()
edgelist.to_csv(output_path+'gene_attribute_edges.txt.gz', sep='\t', compression='gzip')
edgelist

### Gene List

In [ ]:
geneslist = edgelist.get(['Gene', 'Gene ID']).drop_duplicates().reset_index(drop=True)
geneslist.to_csv(output_path+'gene_list_terms.txt.gz', sep='\t', compression='gzip')
geneslist

### Attribute List

In [ ]:
attributeslist = edgelist.get(['Cell Type', 'Cell Type ID']).drop_duplicates().reset_index(drop=True)
attributeslist.to_csv(output_path+'attribute_list_entries.txt.gz', sep='\t', compression='gzip')
attributeslist

### Gene Set Library

In [ ]:
with open(output_path+'gene_set_library_crisp.gmt', 'w') as f:
    arr = binarymatrix.reset_index(drop=True).to_numpy(dtype=np.int_)
    attributes = binarymatrix.columns

    w, h = arr.shape
    for i in tqdm(range(h)):
        if len([*binarymatrix.index[arr[:, i] == 1]])>= 5:
            print(attributes[i], *binarymatrix.index[arr[:, i] == 1], sep='\t', end='\n', file=f)

### Attribute Set Library

In [ ]:
with open(output_path+'attribute_set_library_crisp.gmt', 'w') as f:
    arr = binarymatrixT.reset_index(drop=True).to_numpy(dtype=np.int_)
    genes = binarymatrixT.columns

    w, h = arr.shape
    for i in tqdm(range(h)):
        if len([*binarymatrixT.index[arr[:, i] == 1]])>= 5:
            print(genes[i], *binarymatrixT.index[arr[:, i] == 1], sep='\t', end='\n', file=f)

### Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = dist.pdist(binarymatrix.to_numpy(dtype=np.int_), 'cosine')
gene_similarity_matrix = dist.squareform(gene_similarity_matrix)
gene_similarity_matrix = 1 - gene_similarity_matrix

gene_similarity_matrix = pd.DataFrame(data=gene_similarity_matrix, index=binarymatrix.index, columns=binarymatrix.index)
gene_similarity_matrix.index.name = None
gene_similarity_matrix.columns.name = None
gene_similarity_matrix.to_csv(output_path+'gene_similarity_matrix_cosine.txt.gz', sep='\t', compression='gzip')
gene_similarity_matrix

### Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = dist.pdist(binarymatrixT.to_numpy(dtype=np.int_), 'cosine')
attribute_similarity_matrix = dist.squareform(attribute_similarity_matrix)
attribute_similarity_matrix = 1 - attribute_similarity_matrix

attribute_similarity_matrix = pd.DataFrame(data=attribute_similarity_matrix, index=binarymatrixT.index, columns=binarymatrixT.index)
attribute_similarity_matrix.index.name = None
attribute_similarity_matrix.columns.name = None
attribute_similarity_matrix.to_csv(output_path+'attribute_similarity_matrix_cosine.txt.gz', sep='\t', compression='gzip')
attribute_similarity_matrix

### Knowledge Graph Serialization

In [ ]:
nodes = {}
edges = []

for gene in geneslist.index:
    gene = geneslist.loc[gene]
    nodes[int(gene['Gene ID'])] = {
        "type":"gene",
        "properties": {
            "id":int(gene['Gene ID']),
            "label":gene['Gene']
        }}

for celltype in attributeslist.index:
    celltype = attributeslist.loc[celltype]
    nodes[celltype['Cell Type ID']] = {
        "type":"cell type",
        "properties": {
            "id":celltype['Cell Type ID'].replace('_',':'),
            "label":celltype['Cell Type']
        }}

for edge in edgelist.index:
    edge = edgelist.loc[edge]
    edges.append({
        "source": int(edge['Gene ID']),
        "relation": "is marker for",
        "target": edge['Cell Type ID'].replace('_',':'),
        "properties":{
            "id":str(edge['Gene ID'])+":"+edge['Cell Type ID'].replace('_',':'),
            "source_id":int(edge['Gene ID']),
            "source_label":edge['Gene'],
            "target_label":edge['Cell Type'].replace('_',':'),
            "target_id":edge['Cell Type ID'],
            "directed":True,
            "threshold":1
        }})

#### RDF

In [ ]:
with open(output_path+'kg_serializations/asctbaug.rdf', 'w') as f:
    print('@prefix gene: <https://www.ncbi.nlm.nih.gov/gene/> .', file=f)
    print('@prefix RO: purl.obolibrary.org/RO_', file=f)
    print('@prefix CL: <http://purl.obolibrary.org/obo/CL_> .', file=f)
    print('@prefix PCL: <http://purl.obolibrary.org/obo/PCL_> .', file=f)
    print('@prefix LMHA: <http://purl.obolibrary.org/obo/LMHA_> .', file=f)
    
    print('', file=f)
    for edge in edges:
        print('gene:'+str(edge['properties']['source_id']), 'RO:0002607', edge['properties']['target_id'], end=' .\n', file=f)

#### JSON

In [ ]:
with open(output_path+'kg_serializations/asctbaug.json', 'w') as f:
    serial = json.dump(
        {
            "Version":"1", 
            "nodes": nodes,
            "edges": edges
        }, indent=4, fp=f)

#### TSV

In [ ]:
def namespace(nodeid):
    if 'PCL' in nodeid:
        return 'Provisional Cell Ontology'
    elif 'CL' in nodeid:
        return 'Cell Ontology'
    return 'NCBI Entrez'

nodeframe = pd.DataFrame(nodes).T
nodeframe['id'] = nodeframe['properties'].apply(lambda x: x['id'])
nodeframe['label'] = nodeframe['properties'].apply(lambda x: x['label'])
nodeframe['namespace'] = nodeframe['id'].astype(str).apply(namespace)
#nodeframe['namespace'] = nodeframe['type'].apply(lambda x: {'gene':'NCBI Entrez', 'pathway':'Reactome'}[x])
nodeframe = nodeframe.get(['namespace', 'id', 'label']).reset_index(drop=True)
nodeframe.to_csv(output_path+'kg_serializations/asctbaugmented_tsv/nodes.tsv', sep='\t')
nodeframe

In [ ]:
edgeframe = pd.DataFrame(edges)
edgeframe['threshold'] = edgeframe['properties'].apply(lambda x: x['threshold'])
edgeframe = edgeframe.get(['source', 'relation', 'target', 'threshold'])
edgeframe.to_csv(output_path+'kg_serializations/asctbaugmented_tsv/edges.tsv', sep='\t')
edgeframe

## Create Visualizations

In [ ]:
sns.clustermap(binarymatrix, cmap='seismic', center=0, xticklabels=False, yticklabels=False)

### Gene Similarity Clustered Heatmap

In [ ]:
sns.clustermap(gene_similarity_matrix, cmap='seismic', center=0)

### Attribute Similarity Clustered Heatmap

In [ ]:
sns.clustermap(attribute_similarity_matrix, cmap='seismic', center=0, xticklabels=False, yticklabels=False)

### UMAP

In [ ]:
def load_gmt(file):
    gmt = OrderedDict()
    for line in file:
        term, blank, *geneset = line.strip().split('\t')
        gmt[term] = ' '.join(set(geneset))
    return gmt
libdict = load_gmt(open('augmenteddownloads/gene_set_library_crisp.gmt', 'r'))
scatterdir = 'augmentedimages/'

In [ ]:
def process_scatterplot(libdict, nneighbors=30, mindist=0.1, spread=1.0, maxdf=1.0, mindf=1):
    print("\tTF-IDF vectorizing gene set data...")
    vec = TfidfVectorizer(max_df=maxdf, min_df=mindf)
    X = vec.fit_transform(libdict.values())
    print(X.shape)
    adata = anndata.AnnData(X)
    adata.obs.index = libdict.keys()

    print("\tPerforming Leiden clustering...")
    ### the n_neighbors and min_dist parameters can be altered
    sc.pp.neighbors(adata, n_neighbors=nneighbors, use_rep='X')
    sc.tl.leiden(adata, resolution=1.0)
    sc.tl.umap(adata, min_dist=mindist, spread=spread, random_state=42)

    new_order = adata.obs.sort_values(by='leiden').index.tolist()
    adata = adata[new_order, :]
    adata.obs['leiden'] = 'Cluster ' + adata.obs['leiden'].astype('object')

    df = pd.DataFrame(adata.obsm['X_umap'])
    df.columns = ['x', 'y']

    df['cluster'] = adata.obs['leiden'].values
    df['term'] = adata.obs.index
    df['genes'] = [libdict[l] for l in df['term']]

    return df

In [ ]:
def get_scatter_colors(df):
    clusters = pd.unique(df['cluster']).tolist()
    colors = list(Category20[20])[::2] + list(Category20[20])[1::2]
    color_mapper = {clusters[i]: colors[i % 20] for i in range(len(clusters))}
    return color_mapper

def get_scatterplot(scatterdf):
    df = scatterdf.copy()
    color_mapper = get_scatter_colors(df)
    df['color'] = df['cluster'].apply(lambda x: color_mapper[x])

    hover_emb = HoverTool(name="df", tooltips="""
        <div style="margin: 10">
            <div style="margin: 0 auto; width:300px;">
                <span style="font-size: 12px; font-weight: bold;">Gene Set:</span>
                <span style="font-size: 12px">@gene_set</span>
            <div style="margin: 0 auto; width:300px;">
                <span style="font-size: 12px; font-weight: bold;">Coordinates:</span>
                <span style="font-size: 12px">(@x,@y)</span>
            <div style="margin: 0 auto; width:300px;">
                <span style="font-size: 12px; font-weight: bold;">Cluster:</span>
                <span style="font-size: 12px">@cluster</span>
            </div>
        </div>
    """)
    tools_emb = [hover_emb, 'pan', 'wheel_zoom', 'reset', 'save']

    plot_emb = figure(
        width=1000, 
        height=700, 
        tools=tools_emb
    )

    source = ColumnDataSource(
        data=dict(
            x = df['x'],
            y = df['y'],
            gene_set = df['term'],
            cluster = df['cluster'],
            colors = df['color'],
            label = df['cluster']
        )
    )

    # hide axis labels and grid lines
    plot_emb.xaxis.major_tick_line_color = None
    plot_emb.xaxis.minor_tick_line_color = None
    plot_emb.yaxis.major_tick_line_color = None
    plot_emb.yaxis.minor_tick_line_color = None
    plot_emb.xaxis.major_label_text_font_size = '0pt'
    plot_emb.yaxis.major_label_text_font_size = '0pt' 

    plot_emb.output_backend = "svg"    
    
    plot_emb.title = 'Gene Sets in the HuBMAP ASCT+B Augmented with RNA-seq Coexpression Library'
    plot_emb.xaxis.axis_label = "UMAP_1"
    plot_emb.yaxis.axis_label = "UMAP_2"
    plot_emb.xaxis.axis_label_text_font_style = 'normal'
    plot_emb.xaxis.axis_label_text_font_size = '18px'
    plot_emb.yaxis.axis_label_text_font_size = '18px'
    plot_emb.yaxis.axis_label_text_font_style = 'normal'
    plot_emb.title.align = 'center'
    plot_emb.title.text_font_size = '18px'
    
    s = plot_emb.scatter(
        'x', 
        'y', 
        size = 4, 
        source = source, 
        color = 'colors'
    )
    
    return plot_emb

In [ ]:
## defaults: nneighbors=30, mindist=0.1, spread=1.0, maxdf=1.0, mindf=1
scatter_df = process_scatterplot(libdict, 
     nneighbors=24,
     #mindist=0.1,
     spread=1.5,
     #maxdf=0.9,
     #mindf=2
)

# Display Scatter Plot
plot = get_scatterplot(scatter_df)
show(plot)

In [ ]:
output_file(filename=f"{scatterdir}/asctbaugmented.html", title = 'Gene Sets in the HuBMAP ASCT+B Augmented with RNA-seq Coexpression Library')
save(plot)